the following code is intended to process the road map images

In [12]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import os

In [16]:
datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

images_path = os.path.join('images/road_maps')

## Roadblocks & Next Steps

2/5/21

Issue: gan can't read img_gen because of img_gen's data type (DirectoryIterator). Attempted using img_gen[0], img_gen[0][0], etc., but none worked.

Next Step: investigate mnist.load_data() and determine how it generates the data used in the original gan found in [explore_gan](explore_gan.ipynb)
    

In [43]:
image_size = 150
batch_size = 25

img_gen = datagen.flow_from_directory(
                images_path,  # Source directory for the images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                #class mode set to None because we're only using these images for comparison agaist yet uncreated images
                class_mode=None)

Found 950 images belonging to 1 classes.


In [44]:
img_gen.shape()

AttributeError: 'DirectoryIterator' object has no attribute 'shape'

In [45]:
img_gen.shape()

AttributeError: 'DirectoryIterator' object has no attribute 'shape'

In [48]:
class GAN():
    def __init__(self):
        self.img_rows = 150
        self.img_cols = 150
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = True

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(200, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(400))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(800))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(400))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(200))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=25, sample_interval=50):

        # Load the dataset
        X_train = img_gen

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/generated_roads/%d.png" % epoch)
        plt.close()

In [49]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=25, sample_interval=200)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 22500)             0         
_________________________________________________________________
dense_35 (Dense)             (None, 400)               9000400   
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 400)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 200)               80200     
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 200)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 201       
Total params: 9,080,801
Trainable params: 9,080,801
Non-trainable params: 0
___________________________________________

TypeError: unsupported operand type(s) for /: 'DirectoryIterator' and 'float'